# Latent Space Distribution in dependence of the Scaling of Gaussian Signals

### Import Modules

In [54]:
# external modules
import torch
import matplotlib.pyplot as plt
import numpy as np
import imageio.v3 as iio

# internal modules
from AE_model import AutoEncoder
from dataset_AE import Dataset_1d

## 1) Load the Model

### Parameters for Net Structure and Data Generation

In [55]:
# taken from training procedure

# net parameters
latent_dimension = 2
variable_space = 20
steps = 5 # not needed 

# model params
size = 1000
generations = 10 # reduced in contrast to training
batch_size = 64 # not needed 

In [56]:
# path to file
PATH: str = "trained_models/AE_NN.pth"
# load model 
model = AutoEncoder(latent_dimension=latent_dimension,
                    variable_space=variable_space,
                    steps=steps)
PNN_state_dict = torch.load(PATH)
model.load_state_dict(PNN_state_dict)

<All keys matched successfully>

## 2) Data Generation and Evaluating the Model

In [99]:
# same space as in the training ]
# TODO: see if the net parameterizes well
scales = np.linspace(0.1, 2, 100)

# results stored in a dictionary
results = {}

# iterating through scale values
for scale in scales:
    # store result in dict
    results[scale] = []
    dataset = Dataset_1d(variable_space=variable_space,
                      size=size,
                      generations=generations,
                      random_scaling=scale)
    for gen_idx in range(generations):
        # evaluate latent space for every signal
        results[scale].append(model.encoder(dataset[gen_idx]).detach().numpy())
    results[scale] = np.array(results[scale])


## 3) Plotting the Model

### 3.1) Formation in Animation

In [111]:
scales_normed = scales/scales.max()
for scale_idx, scale in enumerate(scales):
    x = []
    y = []
    for index, latent_space_coordinates in enumerate(results[scale]):
        x.append(latent_space_coordinates[0])
        y.append(latent_space_coordinates[1])

    plt.scatter(x,y, c=np.ones_like(x) * scales_normed[scale_idx])
    plt.title(f"Data for Scale {round(scale, 2)}")
    plt.xlim([-4, 1])
    plt.ylim([-1, 8])
    plt.colorbar()
    plt.clim(vmin=0.0, vmax=1.0)
    plt.savefig(f"frames/scale_{scale}.png")
    plt.clf()

<Figure size 432x288 with 0 Axes>

### Stack the Frames

In [112]:
frames = np.stack([iio.imread(f"frames/scale_{scale}.png") for scale in scales ], axis=0)
iio.imwrite('animations/AE_scale_formation.gif', frames)